In [1]:
%matplotlib notebook

# %qtconsole

import logging
import time
from pprint import pprint
from importlib import reload

from matplotlib import pyplot as plt
from matplotlib import colors, cm
import numpy as np

import qcodes as qc

from qcodes.instrument.parameter import Parameter
from qcodes.dataset.experiment_container import new_experiment
from qcodes.dataset.database import initialise_database

In [ ]:
%qtconsole

# Imports

In [14]:
from pytopo.rf.alazar import acquisition_tools; reload(acquisition_tools)
from pytopo.rf.alazar import awg_sequences; reload(awg_sequences)
from pytopo.rf.alazar import acquisition_controllers; reload(acquisition_controllers)

<module 'pytopo.rf.alazar.acquisition_controllers' from 'd:\\onedrive\\bf2\\code\\pytopo\\pytopo\\rf\\alazar\\acquisition_controllers.py'>

# Init

## load instruments

In [3]:
%run -i D:/OneDrive/BF2/Code/BF2/BF2/init_station_cqed.py

Connected to: TEKTRONIX AWG5208 (serial:B020237, firmware:FV:6.1.0054.0) in 0.63s


## default instrument config

In [4]:
raw_acq = instools.create_inst(acquisition_controllers.RawAcqCtl, 
                               'raw_acq', 'alazar',
                               allocate_samples=1e9, 
                               force_new_instance=True)

post_demod_acq = instools.create_inst(acquisition_controllers.PostDemodCtl, 
                                      'post_demod_acq', 'alazar',
                                      allocate_samples=1e9, 
                                      force_new_instance=True)

post_iq_acq = instools.create_inst(acquisition_controllers.PostIQCtl, 
                                   'post_iq_acq', 'alazar',
                                   allocate_samples=1e9, 
                                   force_new_instance=True)

Allocating 1000000000 elements (2000.0 MB)
Allocating 1000000000 elements (4000.0 MB)
Allocating 1000000000 elements (4000.0 MB)


# Emulation of key experiments

In [5]:
import broadbean as bb
from broadbean.plotting import plotter
from pytopo.awg_sequencing import broadbean as bbtools; reload(bbtools)

ramp = bb.PulseAtoms.ramp
sine = bb.PulseAtoms.sine

In [6]:
chan_map = {
    1 : ['pulse', 'ats_trigger', None]
}
chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

class SinglePulse(bbtools.BroadBeanSequence):
    
    name = 'single_pulse'

    def sequence(self, cycle_time=10e-6, pulse_len=5e-6, pulse_delay=2e-6,
                 pulse_frq=10e6, pulse_amp=0.25):

        trig_len = 0.1e-6
        pre_trig_delay = 0.1e-6
        low_time = cycle_time - pulse_delay - pulse_len - trig_len - pre_trig_delay

        bps = bbtools.BluePrints(chan_map=chan_map, length=cycle_time)

        bps['pulse'].insertSegment(0, ramp, (0,0), dur=pre_trig_delay + trig_len + pulse_delay)
        bps['pulse'].insertSegment(1, sine, (pulse_frq, pulse_amp, 0, 0), name='pulse', dur=pulse_len)
        bps['pulse'].insertSegment(2, ramp, (0, 0), dur=low_time)

        bps['ats_trigger'] = [(pre_trig_delay, trig_len)]

        elt = bbtools.blueprints2element(bps)
        return bbtools.elements2sequence([elt], self.name) 
    

bbseq = SinglePulse(station.awg, chan_map=chan_map, chan_settings=chan_settings)
bbseq.wait = 'off'
bbseq.setup_awg()

Waveforms done
['wfm_1_1_1']
True debug
[['wfm_1_1_1']]


In [7]:
post_demod_acq.demod_frq(10e6)
post_demod_acq.average_buffers(True)

data = acquisition_tools.simple_triggered_sweep_acquisition(
    npts=10, acq_time=8e-6, navgs=1, ctl=post_demod_acq,
)
print(data.shape)

'allocated_buffers' should be <= 'buffers_per_acquisition'. Defaulting 'allocated_buffers' to 1


Setup capture: 0.12288 MB total
 * Buffers: 1 (0.12288 MB/buffer) | (Allocated buffers: 10 = 1.2288000000000001 MB)
 * Records: 10 (0.012288 MB/record)
 * Samples: 1536 (= 7.68 us)
 * Channels: 2
(1, 1, 10, 76, 2)


In [15]:
bogus = Parameter('bogus', set_cmd=None, initial_value=1, unit='A')
nonsense = Parameter('nonsense', set_cmd=None, initial_value=1e3, unit='mV')

hard_axes = [
    ('repetition', np.arange(10)),
    ('time', None),
    ('channel', [1,2])
]

exp = new_experiment(name='testing', sample_name='no sample')

m = acquisition_tools.AlazarMeasurement(
    station, hard_axes=hard_axes, exp=exp, 
    real=False, imag=False, mag=True, phase=True,
    soft_params=[bogus, nonsense, 'crap']
)

with m.run() as datasaver:
    for i in [1,2]:
        for j in [1e3]:
            for k in [-7, -5]:
                result = m.get_result(data, 
                                      (bogus, i), (nonsense, j), ('crap', k), 
                                      ('time', post_demod_acq.demod_tvals))
                datasaver.add_result(*result)

Starting experimental run with id: 18


# Scribble